In [1]:
import os
import torch
from torch import nn
import math
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, DataCollatorWithPadding, TrainingArguments
from dataclasses import dataclass
from transformers.modeling_outputs import ModelOutput
from typing import Optional, Tuple

# Enable CUDA debugging
os.environ["TORCH_USE_CUDA_DSA"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

c:\Users\Administrator\anaconda3\envs\qa_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_files = {
    "train": "train-open.json",
    "validation": "val-open.json",
    "test": "test-open.json"
}
dataset = load_dataset("json", data_files=data_files)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
tokenizer.pad_token = tokenizer.eos_token

# Preprocess Dataset
def preprocess_function(examples):
    inputs = tokenizer(
        examples["question"], max_length=128, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        examples["answer"], max_length=128, truncation=True, padding="max_length"
    )

    inputs["labels"] = labels["input_ids"]
    return inputs

# Tokenize Dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 12715/12715 [00:10<00:00, 1240.00 examples/s]


In [3]:
# Define DifferentialAttention Class
class DifferentialAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.qkv_proj = nn.Linear(d_model, d_model * 3)
        self.lambda_param = nn.Parameter(torch.tensor(0.8))

    def forward(self, x):
        qkv = self.qkv_proj(x).chunk(3, dim=-1)
        q, k, v = qkv
        attention_scores = torch.softmax((q @ k.transpose(-2, -1)) / math.sqrt(k.size(-1)), dim=-1)
        return attention_scores @ v

# Define Custom Output Class
@dataclass
class CustomCausalLMOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

# Define Custom Model Class
class CustomModel(nn.Module):
    def __init__(self, base_model, d_model, num_heads):
        super().__init__()
        self.base_model = base_model
        self.differential_attention = DifferentialAttention(d_model, num_heads)
        self.vocab_projection = nn.Linear(d_model, base_model.config.vocab_size)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.base_model(input_ids, attention_mask=attention_mask, labels=labels, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]  # Last hidden state
        attention_output = self.differential_attention(hidden_states)
        attention_output = self.vocab_projection(attention_output)

        if labels is not None:
            print(f"attention_output shape: {attention_output.shape}")
            print(f"labels shape: {labels.shape}")

            shift_logits = attention_output[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            print(f"shift_logits shape: {shift_logits.shape}")
            print(f"shift_labels shape: {shift_labels.shape}")

            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            return CustomCausalLMOutput(loss=loss, logits=attention_output)

        return CustomCausalLMOutput(logits=attention_output)


In [5]:
# Load Pre-trained Model
base_model = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-base")
base_model.config.output_hidden_states = True  # Enable hidden states

# Define Custom Model
d_model = base_model.config.hidden_size
num_heads = base_model.config.num_attention_heads
model = CustomModel(base_model, d_model, num_heads)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./QAMODELLLLLL",  # Save directory
    eval_strategy="epoch",           # Evaluate at the end of each epoch
    learning_rate=3e-5,              # Adjust learning rate
    per_device_train_batch_size=2,   # Reduce batch size
    num_train_epochs=3,              # Number of epochs
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",            # Directory for logs
    logging_steps=10,
    warmup_steps=100,                # Warmup steps
    gradient_accumulation_steps=4,   # Gradient accumulation
    fp16=False,                      # Disable mixed precision
    no_cuda=False,                   # Enable GPU
    run_name="ffffffinetuning" # Experiment name
)

In [6]:
# Define Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(1000)),  # Subset for training
    eval_dataset=tokenized_datasets["validation"].select(range(200)),  # Subset for evaluation
    data_collator=data_collator
)

# Train the Model
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mohammedberrhazi003 (mohammedberrhazi003-universit-internationale-de-rabat). Use `wandb login --relogin` to force relogin


  0%|          | 0/375 [00:00<?, ?it/s]

attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])


  0%|          | 1/375 [00:25<2:37:31, 25.27s/it]

attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])


...

attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])


100%|█████████▉| 374/375 [2:51:18<00:22, 22.79s/it]

attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])
attention_output shape: torch.Size([2, 128, 64000])
labels shape: torch.Size([2, 128])
shift_logits shape: torch.Size([2, 127, 64000])
shift_labels shape: torch.Size([2, 127])


100%|██████████| 375/375 [2:51:40<00:00, 22.51s/it]

RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'base_model.lm_head.weight', 'base_model.transformer.wte.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

In [7]:
# Evaluate the Model
test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"].select(range(200)))
print(test_results)


attention_output shape: torch.Size([8, 128, 64000])
labels shape: torch.Size([8, 128])
shift_logits shape: torch.Size([8, 127, 64000])
shift_labels shape: torch.Size([8, 127])


attention_output shape: torch.Size([8, 128, 64000])
labels shape: torch.Size([8, 128])
shift_logits shape: torch.Size([8, 127, 64000])
shift_labels shape: torch.Size([8, 127])


attention_output shape: torch.Size([8, 128, 64000])
labels shape: torch.Size([8, 128])
shift_logits shape: torch.Size([8, 127, 64000])
shift_labels shape: torch.Size([8, 127])


attention_output shape: torch.Size([8, 128, 64000])
labels shape: torch.Size([8, 128])
shift_logits shape: torch.Size([8, 127, 64000])
shift_labels shape: torch.Size([8, 127])


attention_output shape: torch.Size([8, 128, 64000])
labels shape: torch.Size([8, 128])
shift_logits shape: torch.Size([8, 127, 64000])
shift_labels shape: torch.Size([8, 127])


...

attention_output shape: torch.Size([8, 128, 64000])
labels shape: torch.Size([8, 128])
shift_logits shape: torch.Size([8, 127, 64000])
shift_labels shape: torch.Size([8, 127])


attention_output shape: torch.Size([8, 128, 64000])
labels shape: torch.Size([8, 128])
shift_logits shape: torch.Size([8, 127, 64000])
shift_labels shape: torch.Size([8, 127])


attention_output shape: torch.Size([8, 128, 64000])
labels shape: torch.Size([8, 128])
shift_logits shape: torch.Size([8, 127, 64000])
shift_labels shape: torch.Size([8, 127])


                                                   
100%|██████████| 375/375 [3:00:41<00:00, 22.51s/it]

{'eval_loss': 1.1760284900665283, 'eval_runtime': 301.8594, 'eval_samples_per_second': 0.663, 'eval_steps_per_second': 0.083, 'epoch': 3.0}
{'eval_loss': 1.1760284900665283, 'eval_runtime': 301.8594, 'eval_samples_per_second': 0.663, 'eval_steps_per_second': 0.083, 'epoch': 3.0}


In [8]:
# Save the Model and Components
model.base_model.save_pretrained("./arabicaqa_model-base22")
tokenizer.save_pretrained("./arabicaqa_model-base22")
torch.save(model.differential_attention.state_dict(), "./arabicaqa_model-base/differential_attention.pth")

In [12]:
# Test the Model
def generate_answer(question):
    # Tokenize the input question
    inputs = tokenizer(question, return_tensors="pt", max_length=128, truncation=True, padding="max_length").to(model.base_model.device)
    
    # Generate a response using max_new_tokens
    outputs = model.base_model.generate(**inputs, max_new_tokens=50)  # Generate up to 50 new tokens
    
    # Decode the output to text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
question ="ما هو عدد سكان إستونيا؟"
answer = generate_answer(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question: ما هو عدد سكان إستونيا؟
Answer: ما هو عدد سكان إستونيا؟وداتينوبيينوبودنا هما من بين أكبر دول حوض النيل ، هما من بين أكبر دول حوض النيل ، هما من بين أكبر دول حوض النيل ، هما من بين أكبر دول حوض النيل ، هما من بين أكبر دول حوض النيل ، هما من بين
